In [1]:
# Import dependencies
import os
import pandas as pd
import gmaps
import requests
# Import the API key
from config import google_API_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame
load_cities = os.path.join('Weather Data', 'cities.csv')
city_data_df = pd.read_csv(load_cities)
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2022-03-10 11:19:10,-23.1203,-134.9692,77.95,71,77,7.81
1,1,Saint-Francois,FR,2022-03-10 11:19:34,46.4154,3.9054,56.48,61,0,3.44
2,2,Chokurdakh,RU,2022-03-10 11:12:28,70.6333,147.9167,-9.24,97,100,5.88
3,3,Butaritari,KI,2022-03-10 11:19:15,3.0707,172.7902,80.01,82,94,14.41
4,4,Hermanus,ZA,2022-03-10 11:19:35,-34.4187,19.2345,80.80,60,10,16.73


In [3]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=google_API_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude
locations = city_data_df[['Lat', 'Lng']]
# Get the maximum temperture
max_temp = city_data_df['Max Temp']
# Assign the figure variable
fig = gmaps.figure(center=(30, 31), zoom_level=1.5)
# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer
fig.add_layer(heat_layer)
# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
humidity = city_data_df['Humidity']
fig2 = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig2.add_layer(heat_layer)
# Call the figure to plot the data
fig2

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
cloudiness = city_data_df['Cloudiness']
fig3 = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, max_intensity=300, point_radius=4)
fig3.add_layer(heat_layer)
# Call the figure to plot the data
fig3

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of wind speed
wind = city_data_df['Wind Speed']
fig4 = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
fig4.add_layer(heat_layer)
# Call the figure to plot the data
fig4

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperture value
min_temp = float(input('What is the minimum temperature(°F) you would like for your trip? '))
max_temp = float(input('What is the maximum temperature(°F) you would like for your trip? '))

What is the minimum temperature(F) you would like for your trip? 75
What is the maximum temperature(F) you would like for your trip? 90


In [10]:
# Filter the dataset to find cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<=max_temp) & (city_data_df['Max Temp']>=min_temp)].copy()
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2022-03-10 11:19:10,-23.1203,-134.9692,77.95,71,77,7.81
3,3,Butaritari,KI,2022-03-10 11:19:15,3.0707,172.7902,80.01,82,94,14.41
4,4,Hermanus,ZA,2022-03-10 11:19:35,-34.4187,19.2345,80.80,60,10,16.73
12,12,Mogadishu,SO,2022-03-10 11:15:01,2.0371,45.3438,84.42,62,100,22.82
14,14,Dutlwe,BW,2022-03-10 11:19:39,-23.9833,23.9000,83.68,37,16,9.10


In [11]:
preferred_cities_df.count()

City_ID       179
City          179
Country       177
Date          179
Lat           179
Lng           179
Max Temp      179
Humidity      179
Cloudiness    179
Wind Speed    179
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,77.95,-23.1203,-134.9692,
3,Butaritari,KI,80.01,3.0707,172.7902,
4,Hermanus,ZA,80.80,-34.4187,19.2345,
12,Mogadishu,SO,84.42,2.0371,45.3438,
14,Dutlwe,BW,83.68,-23.9833,23.9000,
16,Majholi,IN,88.84,23.5000,79.9167,
19,Carnarvon,AU,84.27,-24.8667,113.6333,
22,Puro,PH,77.56,12.4841,123.3827,
24,Saint-Philippe,RE,87.39,-21.3585,55.7679,
29,Vaini,TO,77.16,-21.2000,-175.2000,


In [13]:
# Set parameters to search for a hotel
params = {
    'key':google_API_key,
    'radius':5000,
    'type':'lodging'}

In [21]:
# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    # Add the latitude and longitude to location key for params dictionary
    params['location'] = f'{lat},{lng}'
    # Use the search term: "lodging" and our latitude and longitude
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print(f'Hotel not found for {city}... skipping')
hotel_df.head(10)

Hotel not found for Dutlwe... skipping
Hotel not found for Puro... skipping
Hotel not found for Camacupa... skipping
Hotel not found for Taoudenni... skipping
Hotel not found for Goundam... skipping
Hotel not found for Adrar... skipping
Hotel not found for Thinadhoo... skipping
Hotel not found for Madimba... skipping
Hotel not found for Kidal... skipping
Hotel not found for Poum... skipping
Hotel not found for Wattegama... skipping
Hotel not found for Sapa... skipping


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,77.95,-23.1203,-134.9692,People ThankYou
3,Butaritari,KI,80.01,3.0707,172.7902,Isles Sunset Lodge
4,Hermanus,ZA,80.80,-34.4187,19.2345,Misty Waves Boutique Hotel
12,Mogadishu,SO,84.42,2.0371,45.3438,Hotel Juba
14,Dutlwe,BW,83.68,-23.9833,23.9000,
16,Majholi,IN,88.84,23.5000,79.9167,Amit Jain majholi
19,Carnarvon,AU,84.27,-24.8667,113.6333,Hospitality Carnarvon
22,Puro,PH,77.56,12.4841,123.3827,
24,Saint-Philippe,RE,87.39,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
29,Vaini,TO,77.16,-21.2000,-175.2000,Keleti Beach Resort


In [33]:
info_box_template = '''
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp} °F</dd>
</dl>'''
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [35]:
# Add a heatmap of temperature for the vaction spots and a pop-up marker for each city
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig5 = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig5.add_layer(heat_layer)
fig5.add_layer(marker_layer)
# Call the figure to plot the data
fig5

Figure(layout=FigureLayout(height='420px'))